<a href="https://colab.research.google.com/github/kzmszk/memos/blob/master/ai_economy_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI-Driven Economy Simulation 🚀📈

Experiment with how a sudden “AGI productivity shock” might ripple through key macro indicators.

### How to use
1. Run every cell (⇧+⏎).  
2. Move the sliders to tweak **growth rates**, **shock timing**, or **employment decline**.  
3. The plot updates instantly. Feel free to fork or extend!

> Dependencies: `numpy`, `pandas`, `matplotlib`, `ipywidgets`  (install with `pip install ipywidgets` if missing).

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, HBox, VBox, Layout
%matplotlib inline

In [2]:
def simulate(
    start_year=2025,
    end_year=2040,
    baseline_growth=0.03,
    shock_growth=0.15,
    post_growth=0.05,
    shock_start=2028,
    shock_end=2032,
    shock_emp_decline=-0.01
):
    """Return macro indices over time under simple assumptions."""
    years = np.arange(start_year, end_year + 1)

    # productivity growth path
    growth_series = []
    for y in years:
        if y < shock_start:
            growth_series.append(baseline_growth)
        elif y <= shock_end:
            growth_series.append(shock_growth)
        else:
            growth_series.append(post_growth)

    # productivity / GDP index (2025=1)
    prod_idx = [1.0]
    for g in growth_series:
        prod_idx.append(prod_idx[-1] * (1 + g))
    prod_idx = prod_idx[1:]
    gdp_idx = prod_idx  # 1:1 for toy model

    # simple price level: inverse of productivity
    price_idx = [1 / p for p in prod_idx]

    # employment path
    emp_idx = []
    emp = 1.0
    for y in years:
        if y < shock_start:
            emp_idx.append(emp)
        elif y <= shock_end:
            emp *= (1 + shock_emp_decline)
            emp_idx.append(emp)
        else:
            emp_idx.append(emp)

    # real aggregate wages (proxy)
    real_wage_idx = [p * e for p, e in zip(prod_idx, emp_idx)]

    df = pd.DataFrame({
        "Year": years,
        "ProductivityIdx": prod_idx,
        "GDPIdx": gdp_idx,
        "EmploymentIdx": emp_idx,
        "PriceIdx": price_idx,
        "RealWageIdx": real_wage_idx
    })
    return df

In [6]:
def plot_simulation(
    baseline_growth=0.03,
    shock_growth=0.15,
    post_growth=0.05,
    shock_start=2028,
    shock_end=2032,
    shock_emp_decline=-0.01
):
    df = simulate(
        baseline_growth=baseline_growth,
        shock_growth=shock_growth,
        post_growth=post_growth,
        shock_start=shock_start,
        shock_end=shock_end,
        shock_emp_decline=shock_emp_decline
    )
    # figure
    fig, axes = plt.subplots(1, 3, figsize=(18, 4))
    axes[0].plot(df['Year'], df['GDPIdx'])
    axes[0].set_title('GDP Index'); axes[0].set_xlabel('Year'); axes[0].grid(True)

    axes[1].plot(df['Year'], df['EmploymentIdx'])
    axes[1].set_title('Employment Index'); axes[1].set_xlabel('Year'); axes[1].grid(True)

    axes[2].plot(df['Year'], df['PriceIdx'])
    axes[2].set_title('Price Level Index'); axes[2].set_xlabel('Year'); axes[2].grid(True)

    plt.tight_layout()
    plt.show()

# interactive UI
interact(
    plot_simulation,
    baseline_growth=FloatSlider(min=0, max=0.1, step=0.005, value=0.03, description='Base g'),
    shock_growth=FloatSlider(min=0.0, max=0.5, step=0.01, value=0.10, description='Shock g'),
    post_growth=FloatSlider(min=0, max=0.1, step=0.005, value=0.05, description='Post g'),
    shock_start=IntSlider(min=2026, max=2035, step=1, value=2028, description='Shock start'),
    shock_end=IntSlider(min=2027, max=2038, step=1, value=2032, description='Shock end'),
    shock_emp_decline=FloatSlider(min=-0.05, max=0.0, step=0.001, value=-0.01, description='Emp Δ')
);

interactive(children=(FloatSlider(value=0.03, description='Base g', max=0.1, step=0.005), FloatSlider(value=0.…